In [1]:
%%writefile creation_db.hql

DROP DATABASE IF EXISTS assignmentDB CASCADE;
CREATE DATABASE assignmentDB LOCATION '/user/jovyan/assignmentDB';

Overwriting creation_db.hql


In [2]:
! hive -f creation_db.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 4.05 seconds
OK
Time taken: 0.128 seconds


In [3]:
%%writefile create_exteral_table.hql

USE assignmentDB;
DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external (
    rowID INT,
    postTypeID INT,
    createDate STRING,
    createMonth STRING,
    createYear STRING,
    OwnerUserId INT,
    ParentId INT,
    Score INT,
    FavoriteCount INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' WITH SERDEPROPERTIES (
"input.regex" = '(?=.*\\brow Id=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"(((\\d{4})-\\d{2})-\\d{2}T\\d{2}:\\d{2}:\\d{2}.\\d{3})\")?(?=.*\\bOwnerUserId=\"(\\d+)\")?(?=.*\\bParentId=\"(\\d+)\")?(?=.*\\bScore=\"(-*\\d+)\")(?=.*\\bFavoriteCount=\"(\\d+)\")?.*')
STORED AS TEXTFILE 
LOCATION '/data/stackexchange1000/posts' 
;

Overwriting create_exteral_table.hql


In [4]:
! hive -f create_exteral_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 3.52 seconds
OK
Time taken: 0.058 seconds
OK
Time taken: 0.273 seconds


In [5]:
%%writefile query_exteral_table.hql

USE assignmentDB;
select count(*) from posts_sample_external ;
select * from posts_sample_external LIMIT 10;

Overwriting query_exteral_table.hql


In [6]:
! hive -f query_exteral_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 3.423 seconds
Query ID = jovyan_20201125020715_90d84648-cf68-4919-ba29-d62781e5532c
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1606268104788_0012, Tracking URL = http://172.17.0.21:8088/proxy/application_1606268104788_0012/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1606268104788_0012
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-11-25 02:07:22,682 Stage-1 map = 0%,  reduce = 0%
2020-11-25 02:07:37,977 Stage-1 map = 55%,  r

In [7]:
%%writefile create_managed_table.hql


set hive.exec.dynamic.partition=true;
set hive.exec.max.dynamic.partitions=2048;
set hive.exec.max.dynamic.partitions.pernode=256;
set hive.exec.dynamic.partition.mode=nonstrict;

USE assignmentDB;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample(
    rowID INT,
    postTypeID INT,
    createDate STRING,
    OwnerUserId INT,
    ParentId INT,
    Score INT,
    FavoriteCount INT
)
PARTITIONED BY (createYear STRING, createMonth STRING) 
;

from posts_sample_external
insert overwrite table posts_sample partition (createYear, createMonth) 
select rowID ,    postTypeID ,    createDate ,OwnerUserId ,ParentId, Score,    FavoriteCount, createYear, createMonth

Overwriting create_managed_table.hql


In [8]:
! hive -f create_managed_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 3.457 seconds
OK
Time taken: 0.07 seconds
OK
Time taken: 0.263 seconds
Query ID = jovyan_20201125020754_1de0d28c-7d22-49f5-9530-ca376cfc4ed7
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1606268104788_0013, Tracking URL = http://172.17.0.21:8088/proxy/application_1606268104788_0013/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1606268104788_0013
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-11-25 02:08:00,914 Stage-1 map = 0%,  reduce = 0%
2020-11-25 02:08:16,197 Stage-1 map = 84%,  reduce = 0%, Cumulative CPU 14.33 sec
2020-11-25 02:08:22,293 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 16.74 sec
MapReduce Total cumulative CPU time: 17 seconds 160 msec
Ended Job = job_1606268104788_0013
St

In [9]:
%%writefile query_managed_table.hql

USE assignmentDB;
select count(*) from posts_sample ;
select * from posts_sample LIMIT 10;

Overwriting query_managed_table.hql


In [10]:
! hive -f query_managed_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 3.409 seconds
OK
39431
Time taken: 1.656 seconds, Fetched: 1 row(s)
OK
1394	2	2008-08-04T16:38:03.667	91	1390	16	NULL	2008	2008-08
3543	2	2008-08-06T15:24:00.787	399	3530	37	NULL	2008	2008-08
4521	2	2008-08-07T08:22:27.440	358	66	25	NULL	2008	2008-08
8689	2	2008-08-12T11:23:28.733	1087	8685	1	NULL	2008	2008-08
9062	2	2008-08-12T17:20:41.993	198	6613	1	NULL	2008	2008-08
14671	2	2008-08-18T14:18:22.310	414	14577	1	NULL	2008	2008-08
16307	2	2008-08-19T14:45:07.997	17	16298	2	NULL	2008	2008-08
18780	2	2008-08-20T20:44:27.947	414	18772	0	NULL	2008	2008-08
18929	2	2008-08-20T21:49:23.203	2031	18912	13	NULL	2008	2008-08
19668	2	2008-08-21T11:56:00.593	279	19654	4	NULL	2008	2008-08
Time taken: 0.231 seconds, Fetched: 10 row(s)


In [11]:
%%writefile query.hql

USE assignmentDB;
select createYear, createMonth, count(*) from posts_sample group by createYear, createMonth;

Overwriting query.hql


In [12]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 3.439 seconds
Query ID = jovyan_20201125020859_fbc96286-7351-4549-812c-c5cbcc50a8d1
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1606268104788_0014, Tracking URL = http://172.17.0.21:8088/proxy/application_1606268104788_0014/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1606268104788_0014
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-11-25 02:09:07,051 Stage-1 map = 0%,  reduce = 0%
2020-11-25 02:09:11,190 St